In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *

from overrides import overrides
import warnings

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util


DATA_ROOT = Path("../data/jigsaw")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [8]:
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, SequenceLabelField, LabelField, MetadataField, ArrayField
import csv

max_seq_len = 512

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None, # TODO: Handle mapping from BERT
                 max_seq_len: Optional[int]=max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[str], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = MemoryOptimizedTextField([proc(x) for x in tokens],
                                   self.token_indexers)
        fields = {"tokens": sentence_field}
        
        wl_feats = np.array([[func(w) for func in word_level_features] for w in tokens])
        fields["word_level_features"] = ArrayField(array=wl_feats)
        
        sl_feats = np.array([func(tokens) for func in sentence_level_features])
        fields["sentence_level_features"] = ArrayField(array=sl_feats)

        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            reader = csv.reader(f)
            next(reader)
            for i, line in enumerate(reader):
                if len(line) == 9:
                    _, id_, text, *labels = line
                elif len(line) == 8:
                    id_, text, *labels = line
                else: raise ValueError(f"line has {len(line)} values")
                yield self.text_to_instance(
                    self.tokenizer(text),
                    id_, np.array([int(x) for x in labels]),
                )

In [9]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import WordpieceIndexer, SingleIdTokenIndexer

_spacy_tok = SpacyWordSplitter(language='en_core_web_sm', pos_tags=False).split_words

from allennlp.data.token_indexers import SingleIdTokenIndexer
token_indexer = SingleIdTokenIndexer(
    lowercase_tokens=True,
)
def tokenizer(x: str):
    return [w.text for w in
            _spacy_tok(x.lower())]

In [10]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [11]:
from allennlp.data.fields import TextField, SequenceLabelField, LabelField, MetadataField, ArrayField
import string
alphabet = set(string.ascii_lowercase)

sentence_level_features: List[Callable[[List[str]], float]] = [
#     lambda x: (np.log1p(len(x)) - 3.628) / 1.065, # stat computed on train set
]

word_level_features: List[Callable[[str], float]] = [
    lambda x: 1 if (x.lower() == x) else 0,
    lambda x: len([c for c in x.lower() if c not in alphabet]) / len(x),
]

def proc(x: str) -> str:
    return x.lower()

class MemoryOptimizedTextField(TextField):
    @overrides
    def __init__(self, tokens: List[str], token_indexers: Dict[str, TokenIndexer]) -> None:
        self.tokens = tokens
        self._token_indexers = token_indexers
        self._indexed_tokens: Optional[Dict[str, TokenList]] = None
        self._indexer_name_to_indexed_token: Optional[Dict[str, List[str]]] = None
        # skip checks for tokens
    @overrides
    def index(self, vocab):
        super().index(vocab)
        self.tokens = None # empty tokens

In [39]:
train_ds = (reader.read(DATA_ROOT / fname) for fname in ["debug.csv"])
list(train_ds)

1it [00:00, 830.72it/s]


[[<allennlp.data.instance.Instance at 0x7f6e46d3d898>]]

In [37]:
from allennlp.common.tqdm import Tqdm

d=dict()
for t in train_ds:
    print(t)
    print(type(t))

1it [00:00, 585.80it/s]

<class 'list'>


In [35]:
from allennlp.data.vocabulary import Vocabulary
vocab = Vocabulary.from_instances(train_ds)

0it [00:00, ?it/s]


In [25]:
# vocab should have been saved using
#vocab.save_to_files(DATA_ROOT / "vocab")

To avoid memory errors, restart here and build embedding matrix

In [ ]:
#from allennlp.data.vocabulary import Vocabulary
#vocab = Vocabulary.from_files(DATA_ROOT / "vocab")

In [ ]:
import fastText
ft_model = fastText.load_model(str(DATA_ROOT / "ft_model.bin"))

In [ ]:
with (DATA_ROOT / "ft_model.txt").open("wt") as f:
    for idx, token in vocab.get_index_to_token_vocabulary().items():
        emb = ft_model.get_word_vector(token)
        emb_as_str = " ".join(["%.4f" % x for x in emb])
        f.write(f"{token} {emb_as_str}\n")

In [ ]:
# Anna's reading code